# Loading data into notebook

In [1]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score,  mean_absolute_percentage_error, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    google_path = '/content/drive/My Drive/IDX Exchange/data/'
except:
    google_path = './'

Mounted at /content/drive


In [3]:
df_01 = pd.read_csv(google_path + 'CRMLSSold202408.csv')
df_02 = pd.read_csv(google_path + 'CRMLSSold202409.csv')
df_03 = pd.read_csv(google_path + 'CRMLSSold202410.csv')
df_04 = pd.read_csv(google_path + 'CRMLSSold202411.csv')
df_05 = pd.read_csv(google_path + 'CRMLSSold202412.csv')
df_06 = pd.read_csv(google_path + 'CRMLSSold202501_filled.csv')   # Columns (4) have mixed types. Set dtype option on import or set low_memory=False
df_07 = pd.read_csv(google_path + 'CRMLSSold202502.csv')
df_08 = pd.read_csv(google_path + 'CRMLSSold202503.csv')
df_09 = pd.read_csv(google_path + 'CRMLSSold202504.csv')
df_10 = pd.read_csv(google_path + 'CRMLSSold202505.csv')
df_11 = pd.read_csv(google_path + 'CRMLSSold202506.csv')
df_12 = pd.read_csv(google_path + 'CRMLSSold202507.csv')
df_13 = pd.read_csv(google_path + 'CRMLSSold202508.csv')
df_14 = pd.read_csv(google_path + 'CRMLSSold202509.csv')

/tmp/ipython-input-26609442.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_11 = pd.read_csv(google_path + 'CRMLSSold202506.csv')


In [4]:
df = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_07, df_08, df_09, df_10, df_11, df_12, df_13, df_14], ignore_index=True)

In [5]:
#preferences
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Data Preprocessing

In [6]:
#preprocessing (removing unnecessary columns)
remove_cols = [
    "BuyerAgentAOR",
    "ListAgentAOR",
    "Flooring",
    "WaterfrontYN",
    "BasementYN",
    "OriginalListPrice",
    "ListingKey",
    "ListAgentEmail",
    "ListAgentFirstName",
    "ListAgentLastName",
    "Latitude",
    "Longitude",
    "UnparsedAddress",
    "ListPrice",
    "DaysOnMarket",
    "ListOfficeName",
    "BuyerOfficeName",
    "CoListOfficeName",
    "ListAgentFullName",
    "CoListAgentFirstName",
    "CoListAgentLastName",
    "BuyerAgentMlsId",
    "BuyerAgentFirstName",
    "BuyerAgentLastName",
    "FireplacesTotal",
    "AssociationFeeFrequency",
    "AboveGradeFinishedArea",
    "ListingKeyNumeric",
    "MLSAreaMajor",
    "TaxAnnualAmount",
    "CountyOrParish",
    "MlsStatus",
    "ElementarySchool",
    "AttachedGarageYN",
    "ParkingTotal",
    "BuilderName",
    "LotSizeAcres",
    "SubdivisionName",
    "BuyerOfficeAOR",
    "StreetNumberNumeric",
    "ListingId",
    "City",
    "TaxYear",
    "BuildingAreaTotal",
    "ContractStatusChangeDate",
    "ElementarySchoolDistrict",
    "CoBuyerAgentFirstName",
    "PurchaseContractDate",
    "ListingContractDate",
    "BelowGradeFinishedArea",
    "BusinessType",
    "StateOrProvince",
    "CoveredSpaces",
    "MiddleOrJuniorSchool",
    "FireplaceYN",
    "HighSchool",
    "Levels",
    "LotSizeDimensions",
    "LotSizeArea",
    "MainLevelBedrooms",
    "HighSchoolDistrict",
    "AssociationFee",
    "MiddleOrJuniorSchoolDistrict",
    "latfilled",
    "lonfilled"]

In [7]:
df = df.drop(columns=remove_cols)
df = df[(df["PropertyType"] == "Residential") & (df["PropertySubType"] == "SingleFamilyResidence")]
df = df.drop(columns=["PropertyType", "PropertySubType"])

In [8]:
#filling na values and converting to numerical
df["ViewYN"] = df["ViewYN"].fillna(0)
df["PoolPrivateYN"] = df["PoolPrivateYN"].fillna(0)

living_area_median = df["LivingArea"].median()
df["LivingArea"] = df["LivingArea"].fillna(living_area_median)

year_built_median = df["YearBuilt"].median()
df["YearBuilt"] = df["YearBuilt"].fillna(year_built_median)

bedrooms_median = df["BedroomsTotal"].median()
df["BedroomsTotal"] = df["BedroomsTotal"].fillna(bedrooms_median)

bathrooms_median = df["BathroomsTotalInteger"].median()
df["BathroomsTotalInteger"] = df["BathroomsTotalInteger"].fillna(bathrooms_median)

df["NewConstructionYN"] = df["NewConstructionYN"].fillna(0)

stories_median = df["Stories"].median()
df["Stories"] = df["Stories"].fillna(stories_median)

df["GarageSpaces"] = df["GarageSpaces"].fillna(0)

lot_size_median = df["LotSizeSquareFeet"].median()
df["LotSizeSquareFeet"] = df["LotSizeSquareFeet"].fillna(lot_size_median)

df["ViewYN"] = df["ViewYN"].astype(int)
df["PoolPrivateYN"] = df["PoolPrivateYN"].astype(int)
df["NewConstructionYN"] = df["NewConstructionYN"].astype(int)

df = df.dropna(subset=["PostalCode"])
df["PostalCode"] = df["PostalCode"].str[:5]
df["PostalCode"] = pd.to_numeric(df["PostalCode"])

df["CloseMonth"] = df["CloseDate"].str[5:7]
df["CloseMonth"] = pd.to_numeric(df["CloseMonth"])

df["CloseYear"] = df["CloseDate"].str[:4]
df["CloseYear"] = pd.to_numeric(df["CloseYear"])

df = df.dropna(subset=["ClosePrice"])
df = df.drop(columns="CloseDate")


In [9]:

df = df[(df["ClosePrice"]>=df["ClosePrice"].quantile(.005))&(df["ClosePrice"]<=df["ClosePrice"].quantile(.995))]


In [11]:
# FIND SOME FEATURES TO ENGINEER AND RETRAIN MODELS ON THOSE
# stories or levels

# PPSF (ClosePrice / LivingArea), Bed/Bath ratios, property age
df["ppsf"] = df["ClosePrice"] / df["LivingArea"]
df['BedBathRatio'] = df['BedroomsTotal'] / df['BathroomsTotalInteger']
df['PropertyAge'] = 2025 - df['YearBuilt']

# Replace infinite values in BedBathRatio with the median
median_bedbathratio = df['BedBathRatio'][np.isfinite(df['BedBathRatio'])].median()
df['BedBathRatio'] = df['BedBathRatio'].replace([np.inf, -np.inf], median_bedbathratio)

# Replace infinite values in ppsf with the median
median_ppsf = df['ppsf'][np.isfinite(df['ppsf'])].median()
df['ppsf'] = df['ppsf'].replace([np.inf, -np.inf], median_ppsf)


# Fill remaining NaNs in the BedBathRatio column with the median
median_bedbathratio = df['BedBathRatio'].median()
df['BedBathRatio'] = df['BedBathRatio'].fillna(median_bedbathratio)


In [21]:
features = df.columns
features = features.tolist()
features

['ViewYN',
 'PoolPrivateYN',
 'ClosePrice',
 'LivingArea',
 'YearBuilt',
 'BathroomsTotalInteger',
 'BedroomsTotal',
 'Stories',
 'NewConstructionYN',
 'GarageSpaces',
 'PostalCode',
 'LotSizeSquareFeet',
 'CloseMonth',
 'CloseYear',
 'ppsf',
 'BedBathRatio',
 'PropertyAge']

In [31]:
#split data for testing and training
df_test = df[(df["CloseMonth"] == 9) & (df["CloseYear"] == 2025)]

df_train = df[~((df["CloseMonth"] == 9) & (df["CloseYear"] == 2025))]

#tell what variables to use
X_train = df_train[features]
X_train.drop(columns=["ClosePrice"], inplace=True)
X_train.drop(columns=['LotSizeSquareFeet', 'BathroomsTotalInteger', 'BedroomsTotal', 'YearBuilt'], inplace=True)
y_train = df_train["ClosePrice"]

X_test = df_test[features]
X_test.drop(columns=["ClosePrice"], inplace=True)
X_test.drop(columns=['LotSizeSquareFeet', 'BathroomsTotalInteger', 'BedroomsTotal', 'YearBuilt'], inplace=True)
y_test = df_test["ClosePrice"]

# Model Training
* Random Forest Regressor
* Decision Tree Regressor
* Gradient Boosting Regressor

Before feature engineering:

Random Forest

* R^2: 0.8178689333989163
* RMSE: 391559.30968102627
* MAE:  192922.75967506395
* MAPE %: 15.736694720761694
* Median MAPE: 9.447516949422836

Decision Tree

* R^2: 0.7298655030806165
* RMSE: 476865.1260154378
* MAE:  231044.53350849505
* MAPE %: 18.488991178829266
* Median MAPE: 11.041331016428959


Gradient Boost

* R^2: 0.8609002144775261
* RMSE: 342191.1465196683
* MAE:  170184.43401635057
* MAPE %: 13.952434205273603
* Median MAPE: 9.140252754344651

After feature engineering:
*  After removing original columns

Random Forest
* R^2: 0.9981381261336466
  * R^2: 0.9978800274835695
* RMSE: 39589.55924915554
  * RMSE: 42244.546675068275
* MAE:  3579.453538757216
  * MAE:  3595.3783275001933
* MAPE %: 0.20133631177640918
  * MAPE %: 0.2014311260626751
* Median MAPE: 0.09594350178905402
  * Median MAPE: 0.09254798919146327

Decision Tree
* R^2: 0.9976132811547291
  * R^2: 0.9969119517022238
* RMSE: 44823.541784909256
  * RMSE: 50985.605440620544
* MAE:  9042.839603865888
  * MAE:  8920.030320229707
* MAPE %: 0.5959072639224359
  * MAPE %: 0.58629956600174
* Median MAPE: 0.3487581440992729
  * Median MAPE: 0.338332639939564

Gradient Boost

* R^2: 0.9983462540644429
  * R^2: 0.9975613650005491
* RMSE: 37311.261565687106
  * RMSE: 45308.42232622969
* MAE:  4873.683535344068
  * MAE:  5107.709147846665
* MAPE %: 0.35722095705008783
  * MAPE %: 0.37038526886824646
* Median MAPE: 0.2374543217335906
  * Median MAPE: 0.23928272792398442

In [32]:
#training Random Forest Model
model = RandomForestRegressor(
    n_estimators = 500,
    max_depth=None,
    random_state=67,
    min_samples_split=2,
    min_samples_leaf=2,
    n_jobs=-1
)


In [33]:

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("R^2:", r2_score(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ", mae)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE %:", mape * 100)
mape_values = np.abs((y_test - y_pred)/y_test) * 100
median_mape = np.median(mape_values)
print("Median MAPE:", median_mape)

R^2: 0.9978800274835695
RMSE: 42244.546675068275
MAE:  3595.3783275001933
MAPE %: 0.2014311260626751
Median MAPE: 0.09254798919146327


In [34]:

model = DecisionTreeRegressor(
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=67
)


In [35]:

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R^2:", r2_score(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ", mae)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE %:", mape * 100)
mape_values = np.abs((y_test - y_pred)/y_test) * 100
median_mape = np.median(mape_values)
print("Median MAPE:", median_mape)

R^2: 0.9969119517022238
RMSE: 50985.605440620544
MAE:  8920.030320229707
MAPE %: 0.58629956600174
Median MAPE: 0.338332639939564


In [36]:

model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.1,
    max_depth=10,
    subsample=.8,
    random_state=67
)

In [37]:


model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R^2:", r2_score(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ", mae)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE %:", mape * 100)
mape_values = np.abs((y_test - y_pred)/y_test) * 100
median_mape = np.median(mape_values)
print("Median MAPE:", median_mape)

R^2: 0.9975613650005491
RMSE: 45308.42232622969
MAE:  5107.709147846665
MAPE %: 0.37038526886824646
Median MAPE: 0.23928272792398442


In [38]:
#training diff models
#model = LinearRegression()




#Gradient Boosting Regressor: R^2: 0.84549811013036, RMSE: 362255.6531145345, MAPE %: 14.180844539661678, Median MAPE: 8.842370969370764
#Random Forest Regressor results: R^2: 0.7981733729751737, RMSE: 414035.42832625436, MAPE %: 16.151178240782073, Median MAPE: 9.633472679394355
#DecisionTreeRegressor results: R^2: 0.6520970912127946, RMSE: 543597.3155486308, MAPE %: 26.505989890415556, Median MAPE: 17.966200485594783
#Linear Regressor results: R^2: 0.42680043995468564, RMSE: 697752.1729472455, MAPE %: 46.56291267159915, Median MAPE: 31.32234081762372
print("R^2:", r2_score(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ", mae)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE %:", mape * 100)
mape_values = np.abs((y_test - y_pred)/y_test) * 100
median_mape = np.median(mape_values)
print("Median MAPE:", median_mape)

R^2: 0.9975613650005491
RMSE: 45308.42232622969
MAE:  5107.709147846665
MAPE %: 0.37038526886824646
Median MAPE: 0.23928272792398442
